<a href="https://colab.research.google.com/github/N3wBaz/MLzoomcamp/blob/main/hw3/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('housing.csv')

In [3]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [4]:
df.fillna(0, inplace=True)

In [5]:
df['rooms_per_household'] = df['total_rooms'] / df['households']

In [6]:
df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']

In [7]:
df['population_per_household'] = df['population'] / df['households']

In [8]:
df['ocean_proximity'].mode()

0    <1H OCEAN
dtype: object

In [9]:
# for i in df.columns:
# #     print(i)
#     if i == 'ocean_proximity' or i == 'longitude':
#         continue
#     print(f"{(df.corr()[i]).sort_values()[-2:-1]}")

In [10]:
(df.corr()['total_bedrooms']).sort_values()

housing_median_age         -0.317063
latitude                   -0.065318
population_per_household   -0.028019
median_income              -0.007295
rooms_per_household         0.002717
median_house_value          0.049148
longitude                   0.068082
bedrooms_per_room           0.122205
population                  0.866266
total_rooms                 0.920196
households                  0.966507
total_bedrooms              1.000000
Name: total_bedrooms, dtype: float64

In [11]:
(df.corr()['population']).sort_values()

housing_median_age         -0.296244
latitude                   -0.108785
rooms_per_household        -0.072213
median_house_value         -0.024650
median_income               0.004834
bedrooms_per_room           0.031397
population_per_household    0.069863
longitude                   0.099773
total_rooms                 0.857126
total_bedrooms              0.866266
households                  0.907222
population                  1.000000
Name: population, dtype: float64

In [12]:
(df.corr()['population_per_household']).sort_values()

total_bedrooms             -0.028019
households                 -0.027309
total_rooms                -0.024581
median_house_value         -0.023737
rooms_per_household        -0.004852
latitude                    0.002366
longitude                   0.002476
bedrooms_per_room           0.003047
housing_median_age          0.013191
median_income               0.018766
population                  0.069863
population_per_household    1.000000
Name: population_per_household, dtype: float64

In [13]:
def above_median(series):
    
    return 1 if series.median() > series else 0
# df[]np.where(df['median_house_value'] > df['median_house_value'])

In [14]:
df['above_average'] = (df['median_house_value'].mean() > df['median_house_value']).astype(int)

In [15]:
df['above_average']

0        0
1        0
2        0
3        0
4        0
        ..
20635    1
20636    1
20637    1
20638    1
20639    1
Name: above_average, Length: 20640, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
SEED = 42
df_train, df_test_val = train_test_split(df, test_size=0.4, random_state=SEED)
df_test, df_val = train_test_split(df_test_val, test_size=0.5, random_state=SEED)

In [18]:
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)


y_train = df_train['median_house_value'].values
y_test = df_test['median_house_value'].values
y_val = df_val['median_house_value'].values



# y_train = y_train.reset_index(drop=True)



In [19]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

In [20]:
df_train['above_average'].mean()

0.596172480620155

In [21]:
from sklearn.metrics import mutual_info_score

In [22]:
round(mutual_info_score(df_train['above_average'], df_train['ocean_proximity']), 3)

0.103

In [40]:
# from sklearn.preprocessing import OneHotEncoder

# one_hot_train = pd.get_dummies(df_train, columns = ['ocean_proximity'], drop_first=True)
# one_hot_val = pd.get_dummies(df_val, columns = ['ocean_proximity'], drop_first=True)
# one_hot_test = pd.get_dummies(df_test, columns = ['ocean_proximity'], drop_first=True)


(4128, 57357)

In [34]:
# print(one_hot_train.columns)
# print(one_hot_val.columns)
# print(one_hot_test.columns)

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'rooms_per_household', 'bedrooms_per_room', 'population_per_household',
       'above_average', 'ocean_proximity_INLAND', 'ocean_proximity_ISLAND',
       'ocean_proximity_NEAR BAY', 'ocean_proximity_NEAR OCEAN'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'rooms_per_household', 'bedrooms_per_room', 'population_per_household',
       'above_average', 'ocean_proximity_INLAND', 'ocean_proximity_NEAR BAY',
       'ocean_proximity_NEAR OCEAN'],
      dtype='object')
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'rooms_per_household', 'bedrooms_per_room', 'population_per_household',
       'above_average', 'ocean_proximity_INL

In [25]:
# one_hot_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12384 entries, 0 to 12383
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   longitude                   12384 non-null  float64
 1   latitude                    12384 non-null  float64
 2   housing_median_age          12384 non-null  float64
 3   total_rooms                 12384 non-null  float64
 4   total_bedrooms              12384 non-null  float64
 5   population                  12384 non-null  float64
 6   households                  12384 non-null  float64
 7   median_income               12384 non-null  float64
 8   rooms_per_household         12384 non-null  float64
 9   bedrooms_per_room           12384 non-null  float64
 10  population_per_household    12384 non-null  float64
 11  above_average               12384 non-null  int64  
 12  ocean_proximity_INLAND      12384 non-null  uint8  
 13  ocean_proximity_ISLAND      123

In [41]:
from sklearn.feature_extraction import DictVectorizer

In [42]:
numerical = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
             'total_bedrooms', 'population', 'households', 'median_income',
             'rooms_per_household', 'bedrooms_per_room', 'population_per_household', 'above_average']
categorical = ['ocean_proximity']

In [87]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[numerical + categorical].to_dict(orient='records')
test_dict = df_test[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dict)
X_test = dv.transform(test_dict)

In [88]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [89]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [90]:
model = LogisticRegression(solver="liblinear", C=1.0, max_iter=100, random_state=SEED)
model.fit(X_train, y_train)
# y_train.shape

LogisticRegression(random_state=42, solver='liblinear')

In [93]:
model.predict(X_train)

array([448100.,  53800., 111300., ..., 222100., 283500., 325000.])

In [56]:
from sklearn.metrics import accuracy_score

array([[0.00000000e+00, 1.81818182e-01, 6.53000000e+02, ...,
        5.81163859e+00, 6.90000000e+02, 3.79500000e+03],
       [1.00000000e+00, 1.91666667e-01, 2.40000000e+01, ...,
        1.00000000e+01, 4.60000000e+01, 2.40000000e+02],
       [1.00000000e+00, 2.07034939e-01, 8.81000000e+02, ...,
        4.80817253e+00, 8.77000000e+02, 4.23600000e+03],
       ...,
       [0.00000000e+00, 2.70823417e-01, 5.27000000e+02, ...,
        3.98671727e+00, 5.69000000e+02, 2.10100000e+03],
       [0.00000000e+00, 1.66993007e-01, 5.59000000e+02, ...,
        6.39534884e+00, 5.97000000e+02, 3.57500000e+03],
       [0.00000000e+00, 3.11168954e-01, 1.24200000e+03, ...,
        3.40257649e+00, 1.31500000e+03, 4.22600000e+03]])

In [114]:
round(accuracy_score(y_val, model.predict(X_val)), 2)

0.05